## Trabajo práctico 2
### Alumnos: Francisco Frusto Alvarado, Ezequiel Kaplan

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [124]:
#Inicializacion de pesos
W1 = np.array(np.random.random((5,6)))
b1 = np.array(np.random.random((5,1)))

W2 = np.array(np.random.random((1,5)))
b2 = np.array(np.random.random((1,1)))

theta = np.array([W1, b1, W2, b2])
print(theta)


[array([[0.87042279, 0.32430587, 0.21274525, 0.68317046, 0.82073688,
         0.41763618],
        [0.89249721, 0.22039439, 0.8341225 , 0.04978126, 0.57075423,
         0.94614074],
        [0.98854873, 0.38136489, 0.65550829, 0.51858125, 0.31052127,
         0.74015276],
        [0.91538638, 0.2058532 , 0.16920507, 0.5432499 , 0.80571925,
         0.33677357],
        [0.53106099, 0.24648787, 0.42937119, 0.55128718, 0.06002364,
         0.96753509]])
 array([[0.9826956 ],
        [0.61303429],
        [0.09561553],
        [0.00664816],
        [0.91244399]])
 array([[0.37263231, 0.9865202 , 0.83292144, 0.85717883, 0.63060507]])
 array([[0.1346366]])]


/var/folders/5d/4g206gss6cg3fktfbycqnb540000gn/T/ipykernel_27631/1755964871.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  theta = np.array([W1, b1, W2, b2])


Para implementar la funcion forward, vamos a hacer los cálculos de a pasos, sabiendo:

$\newline f_{\theta}(\mathrm{\mathbf{x}}) = W_{2} \ \sigma (W_{1} \mathrm{\mathbf{x}} + b_{1}) + b{2} $

En una primera instancia calculamos $ z_{1} = W_{1} \mathrm{\mathbf{x}} + b_{1}$ en la que hacemos el producto punto entre $\mathrm{\mathbf{x}}$ y $W_{1}$ y luego le sumamos el vector $b_{1}$. 

En un segundo paso, habiendo creado la funcion sigmoid que calcula: $\sigma(x) = \frac{1}{1 + e^{-x}}$ vamos a aplicarle esta funcion a los elementos de $z_{1}$

En un tercer paso, calculamos el resultado final en el que se hace el producto punto entre $W_{2}$ y el resultado anterior y ademas, se le suma el vector $b_{2}$

In [125]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def forward(theta, x):
    W1, b1, W2, b2 = theta
    x = np.array(x)
    x = x[:, np.newaxis]

    z1 = np.dot(W1, x) + b1  
    a1 = sigmoid(z1)
    z2 = np.dot(W2, a1) + b2

    return z2

La funcion obejtivo que buscamos minimizar, sabemos que está dada por: 
$\newline L = \frac{1}{2}(f_{\theta_{t}}(\mathrm{\mathbf{x}}_{i}) - y_{i})^2$

Para calcular el gradiente de forma numércia, vamos a usar la estrategia propuesta por la cátedra para calcular las derivadas parciales. 

In [129]:
#Calculo del gradiente numerico

def funcion_objetivo(theta, x, y):
    print(f'theta: {theta}')
    loss = 0.5 * (forward(theta, x) - y)**2
    return loss

def numerical_gradient(theta, x, y, eps):
    epsilon = eps  # Pequeña cantidad para el cálculo numérico del gradiente
    gradiente = [np.zeros_like(param) for param in theta]

    for i in range(len(theta)):
        # Crear copias independientes de theta_plus y theta_minus
        theta_plus = np.array(np.copy(theta))
        theta_minus = np.array(np.copy(theta))
    
        # Aumentar y disminuir un poco el parámetro actual para calcular el gradiente
        theta_plus[i] += epsilon
        theta_minus[i] -= epsilon
        print(f'theta_plus: {theta_plus}')
        print(f'theta_minus: {theta_minus}')

        # Calcular las pérdidas para los parámetros aumentados y disminuidos
        loss_plus = funcion_objetivo(theta_plus, x, y)
        loss_minus = funcion_objetivo(theta_minus, x, y)

        # Calcular el gradiente parcial utilizando las derivadas parciales
        gradiente[i] = (loss_plus - loss_minus) / (2 * epsilon)

    gradiente = np.array(gradiente)

    return gradiente




In [127]:
#funcion fit y loop de entrenamiento
def fit(theta, x, y, learning_rate=0.001, epochs=1000):
    TOLERANCIA = 0.0001
    eps = 1e-1
    loss_accum = []

    for epoch in range(epochs):
        loss_epoch = 0.0  # Pérdida acumulada en el epoch actual

        for i in range(len(x)):
            # Obtener el vector x y la salida esperada y correspondientes
            x_i = x[i]
            y_i = y[i]

            # Calcular el gradiente numérico para el ejemplo actual
            gradient = numerical_gradient(theta, x_i, y_i, eps)
    
            print(f'gradiente: {gradient}, theta: {theta}')

            # Actualizar los parámetros
            theta -= learning_rate * gradient

            # Calcular la pérdida para el ejemplo actual
            loss_i = funcion_objetivo(theta, x_i, y_i)
            loss_epoch += loss_i

        # Calcular la pérdida promedio para el epoch actual
        loss_avg = loss_epoch / len(x)
        loss_accum.append(loss_avg)

        if abs(theta - loss_avg) < TOLERANCIA:
            break

    return loss_accum, theta




In [83]:
def predict(x, theta):
        y = np.dot(x, theta)
        return y

In [130]:
# Entrenar la red neuronal
df_test = pd.read_excel("datos.xlsx", nrows=10)
x = df_test.iloc[:, 1:7].values.tolist()
y = df_test.iloc[:, 7].values.tolist()


loss_accum, theta_new = fit(theta, x, y, learning_rate=0.001, epochs=1000)

# Obtener predicciones en el conjunto de datos de prueba
y_pred = predict(x, theta_new)

# Calcular el error cuadrático medio en el conjunto de datos de prueba
mse = ((y_pred - y) ** 2).mean()

# Graficar la función objetivo a lo largo del entrenamiento
plt.plot(loss_accum)
plt.xlabel('Épocas')
plt.ylabel('Función Objetivo')
plt.title('Evolución de la función objetivo durante el entrenamiento')
plt.show()

# Graficar el error cuadrático medio en el conjunto de datos de prueba
plt.scatter(y, y_pred)
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Predicciones vs. Valores Reales en el conjunto de datos de prueba')
plt.show()

print(f'Error cuadrático medio en el conjunto de datos de prueba: {mse}')


theta_plus: [array([[0.87042279, 0.32430587, 0.21274525, 0.68317046, 0.82073688,
         0.41763618],
        [0.89249721, 0.22039439, 0.8341225 , 0.04978126, 0.57075423,
         0.94614074],
        [0.98854873, 0.38136489, 0.65550829, 0.51858125, 0.31052127,
         0.74015276],
        [0.91538638, 0.2058532 , 0.16920507, 0.5432499 , 0.80571925,
         0.33677357],
        [0.53106099, 0.24648787, 0.42937119, 0.55128718, 0.06002364,
         0.96753509]])
 array([[0.9826956 ],
        [0.61303429],
        [0.09561553],
        [0.00664816],
        [0.91244399]])
 array([[0.37263231, 0.9865202 , 0.83292144, 0.85717883, 0.63060507]])
 array([[0.1346366]])]
theta_minus: [array([[0.87042279, 0.32430587, 0.21274525, 0.68317046, 0.82073688,
         0.41763618],
        [0.89249721, 0.22039439, 0.8341225 , 0.04978126, 0.57075423,
         0.94614074],
        [0.98854873, 0.38136489, 0.65550829, 0.51858125, 0.31052127,
         0.74015276],
        [0.91538638, 0.2058532 , 0.169205

ValueError: non-broadcastable output operand with shape (4,) doesn't match the broadcast shape (4,1,4)